In [1]:
import dgl
import torch
import numpy as np
import pandas as pd
import pickle

import matplotlib.pyplot as plt
import networkx as nx

Using backend: pytorch


In [ ]:
# # vertex/ node column

# df_Category1 = pd.read_excel("Node.xlsx")#, usecols=col_list)
# print('df_Category1: ',df_Category1.shape)
# df_Category1.head(2)

In [2]:
# edges column

col_list = ["~id", "~from", "~to"]
df_edges = pd.read_csv("is-a-1.csv", usecols=col_list)
df_edges = df_edges.fillna(0)
print('df_edges shape: ', df_edges.shape)
#df_edges = df_edges
df_edges.head(2)

df_edges shape:  (393928, 3)


,~id,~from,~to
0,9ab7cf7c-3e5f-0950-624a-028c80b2f3e1,0,23726.0
1,b0b7cf7c-3e6c-5446-615a-f1468af83e34,1,23717.0


In [3]:
# edges column

col_list = ["~id", "~from", "~to"]
df_edges = pd.read_csv("is-a-1.csv", usecols=col_list)
df_edges = df_edges.dropna(axis=0)
print('df_edges shape: ', df_edges.shape)
#df_edges = df_edges
df_edges.head(2)

df_edges shape:  (393927, 3)


,~id,~from,~to
0,9ab7cf7c-3e5f-0950-624a-028c80b2f3e1,0,23726.0
1,b0b7cf7c-3e6c-5446-615a-f1468af83e34,1,23717.0


In [4]:
print('from column max Value: ', max(df_edges['~from']))
print('to column max Value: ', max(df_edges['~to']))

from column max Value:  742247
to column max Value:  392081.0


In [5]:
node_count = 742248
# node_count = 715505

In [6]:
dst = df_edges['~to']
src = df_edges['~from']

In [7]:
edge_list = list(zip(dst, src))
edge_list

[(23726.0, 0),
 (23717.0, 1),
 (23717.0, 2),
 (23717.0, 3),
 (23717.0, 4),
 (23717.0, 6),
 (23717.0, 7),
 (23717.0, 8),
 (23717.0, 10),
 (23717.0, 11),
 (23717.0, 14),
 (23726.0, 15),
 (23726.0, 16),
 (23705.0, 18),
 (23705.0, 19),
 (23705.0, 22),
 (23705.0, 23),
 (23666.0, 24),
 (23666.0, 26),
 (23666.0, 27),
 (23707.0, 30),
 (23707.0, 31),
 (23707.0, 32),
 (23727.0, 34),
 (23727.0, 35),
 (23727.0, 36),
 (23726.0, 38),
 (23633.0, 39),
 (23668.0, 40),
 (23668.0, 41),
 (23668.0, 42),
 (23668.0, 43),
 (23726.0, 44),
 (23677.0, 46),
 (23677.0, 47),
 (23677.0, 48),
 (23677.0, 49),
 (23692.0, 50),
 (23703.0, 52),
 (23700.0, 55),
 (23717.0, 56),
 (23700.0, 57),
 (23700.0, 58),
 (23700.0, 60),
 (23703.0, 61),
 (23703.0, 63),
 (23700.0, 64),
 (23700.0, 65),
 (23703.0, 68),
 (23700.0, 69),
 (23700.0, 71),
 (23703.0, 72),
 (23703.0, 77),
 (23703.0, 85),
 (23703.0, 93),
 (23703.0, 99),
 (23700.0, 100),
 (23703.0, 101),
 (23703.0, 105),
 (23703.0, 107),
 (23703.0, 108),
 (23700.0, 109),
 (23700.0,

In [8]:
import pickle
with open('edge_list.pickle_2', 'wb') as f:
  pickle.dump(edge_list, f)

In [9]:
# Load from pickle file

with open('edge_list.pickle_2', 'rb') as f:
    edge_list = pickle.load(f)

In [ ]:
#print(edge_list)

In [10]:
def build_karate_club_graph(edges):
    g = dgl.DGLGraph()
    g.add_nodes(node_count)
    src, dst = tuple(zip(*edges))
    g.add_edges(src, dst)
    # edges are directional in DGL; make them bidirectional
    g.add_edges(dst, src)
    return g

In [11]:
G = build_karate_club_graph(edge_list)

In [12]:
print('We have %d nodes.' % G.number_of_nodes())
print('We have %d edges.' % G.number_of_edges())

We have 742248 nodes.
We have 787854 edges.


In [13]:
# Since the actual graph is undirected, we convert it for visualization purpose.
nx_G = G.to_networkx().to_undirected()
# Kamada-Kawaii layout usually looks pretty for arbitrary graphs
pos = nx.kamada_kawai_layout(nx_G)
nx.draw(nx_G, pos, with_labels=True, node_color=[[.7, .7, .7]])

MemoryError: 

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# Define the message and reduce function
# NOTE: We ignore the GCN's normalization constant c_ij for this tutorial.
def gcn_message(edges):
    # The argument is a batch of edges.
    # This computes a (batch of) message called 'msg' using the source node's feature 'h'.
    return {'msg' : edges.src['h']}

def gcn_reduce(nodes):
    # The argument is a batch of nodes.
    # This computes the new 'h' features by summing received 'msg' in each node's mailbox.
    return {'h' : torch.sum(nodes.mailbox['msg'], dim=1)}

# Define the GCNLayer module
class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g, inputs):
        # g is the graph and the inputs is the input node features
        # first set the node features
        g.ndata['h'] = inputs
        # trigger message passing on all edges
        g.send(g.edges(), gcn_message)
        # trigger aggregation at all nodes
        g.recv(g.nodes(), gcn_reduce)
        # get the result node features
        h = g.ndata.pop('h')
        # perform linear transformation
        return self.linear(h)

In [ ]:
class GCN(nn.Module):
    def __init__(self, in_feats, hidden_size, num_classes):
        super(GCN, self).__init__()
        self.gcn1 = GCNLayer(in_feats, hidden_size)
        self.gcn2 = GCNLayer(hidden_size, num_classes)
        self.softmax = nn.Softmax()

    def forward(self, g, inputs):
        h = self.gcn1(g, inputs)
        h = torch.relu(h)
        h = self.gcn2(g, h)
        h = self.softmax(h)
        return h
# The first layer transforms input features of size of 34 to a hidden size of 5.
# The second layer transforms the hidden layer and produces output features of
# size 2, corresponding to the two groups of the karate club.
net = GCN(node_count, 5, 2)

In [ ]:
inputs = torch.eye(node_count)
labeled_nodes = torch.tensor([0, node_count])  # only the instructor and the president nodes are labeled
labels = torch.tensor([0,1])  # their labels are different

In [ ]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.1)
all_preds = []
epochs = 2

for epoch in range(epochs):
    preds = net(G, inputs)
    all_preds.append(preds)
    # we only compute loss for labeled nodes
    loss = F.cross_entropy(preds[labeled_nodes], labels)

    optimizer.zero_grad() # PyTorch accumulates gradients by default
    loss.backward() 
    optimizer.step()

    print('Epoch %d | Loss: %.4f' % (epoch, loss.item()))

In [ ]:
last_epoch = all_preds[epochs-1].detach().numpy()
predicted_class = np.argmax(last_epoch, axis=-1)
color = np.where(predicted_class==0, 'c', 'r')

print(predicted_class)
print(color)

In [ ]:
nx.draw_networkx(nx_G, pos, node_color=color, with_labels=True, node_size=300)

In [ ]:
print(last_epoch[8])